This notebook implements the [Assignment 4](https://github.com/sprintml/tml_2024/blob/main/Assignment4.pdf) - Task 3 of Trustworthy Machine Learning course offered in the Summer Semester 2024 at the Saarland University. This task focuses on comparing LIME and Grad CAM methods using IoU metric. The report analyzing the results of this task can be accessed [here](https://github.com/nupur412/TML_Assignment4_Explainability/blob/main/TML_Task_4_Report.pdf)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import cv2
import numpy as np
import os, json

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

Using Tasks 2 and 3, we obtain masks for both LIME and Grad CAM methods

In [ ]:
lime_masks = ['/content/limeMask_0.png', '/content/limeMask_1.png', '/content/limeMask_2.png', '/content/limeMask_3.png',
              '/content/limeMask_4.png', '/content/limeMask_5.png', '/content/limeMask_6.png',
              '/content/limeMask_7.png', '/content/limeMask_8.png', '/content/limeMask_9.png',
              ]

gradcam_masks = ['/content/gradMask_0.png', '/content/gradMask_1.png', '/content/gradMask_2.png',
              '/content/gradMask_3.png', '/content/gradMask_4.png', '/content/gradMask_5.png',
              '/content/gradMask_6.png', '/content/gradMask_7.png', '/content/gradMask_8.png',
              '/content/gradMask_9.png']

In [ ]:
! pip install numpy matplotlib opencv-python pillow

In [ ]:
def process_image(img, target_shape):
    img = img.resize(target_shape, Image.NEAREST)
    img_gray = img.convert('L')
    img_array = np.array(img_gray)

    _, binary_mask = cv2.threshold(img_array, 127, 255, cv2.THRESH_BINARY)
    return binary_mask

In [ ]:
def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

In [ ]:
target_shape = (369, 369)

# convert the masks to binary format
lime_masks = [process_image(get_image(path), target_shape) for path in lime_masks]
gradcam_masks = [process_image(get_image(path), target_shape) for path in gradcam_masks]

Defining the function to compute IoU for masks from LIME and GradCAM

In [ ]:
def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    iou = intersection / union if union != 0 else 0
    return iou

In [ ]:
# Compute IoU for each pair of masks
ious = []
for lime_mask, gradcam_mask in zip(lime_masks, gradcam_masks):
    iou = compute_iou(lime_mask, gradcam_mask)
    ious.append(iou)

print("IoUs:", ious)
print("Average IoU:", np.mean(ious))